# Libs and SOS instance

In [ ]:
from sos4py.main import connection_sos
from sos4py.sos_2_0_0 import SOSGetFeatureOfInterestResponse

from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pandas as pd

# import folium
# import contextily as ctx

In [ ]:
# Create SOS instance 
fluggs_sos = connection_sos("https://fluggs.wupperverband.de/sos2/service")

# 1) Learn about content of SOS 

#### Service

In [ ]:
print(fluggs_sos.sosServiceIdentification())
#print(fluggs_sos.sosServiceIdentification()['profiles'])

#### Provider

In [ ]:
print(fluggs_sos.sosProvider())

#### SOS operations

In [ ]:
print(fluggs_sos.sosOperationsMetadata())
#print(fluggs_sos.sosOperationsMetadata()[1]['Name'])

#### All offerings

In [ ]:
print(fluggs_sos.sosOfferings())
#print(fluggs_sos.sosOfferings()[1])
#print(fluggs_sos.sosOfferings()[1]['bbox'])

#### All available phenomena (independent of e.g feature of interest or offering)

In [ ]:
print(fluggs_sos.sosPhenomena())

#### All available features of interest

In [ ]:
print(fluggs_sos.sosFeaturesOfInterest())

#### Data availability

In [ ]:
#print(fluggs_sos.get_data_availability())

props = ['Wassertemperatur']
fois = ['Opladen','Bever-Talsperre']
gda = fluggs_sos.get_data_availability(observedProperties=props,featuresOfInterest=fois)
print(gda)

# 2) Spatial data

### 2a) Get spatial data

In [ ]:
include_phenomena = False
fluggs_sites = fluggs_sos.get_sites(include_phenomena)
fluggs_sites.head()

In [ ]:
# Check and change coordinate reference system
print(fluggs_sites.crs,"\n")
fluggs_sites = fluggs_sites.to_crs('epsg:4326')

# View specific site using integer-locaiton
print(fluggs_sites.iloc[0])

### 2b) Plot spatial data

In [ ]:
# Static map using GeoDataframe.plot

# sites = sites.to_crs('epsg:3857') # epsg:4326
ax = fluggs_sites.plot(figsize=(15, 15))
ctx.add_basemap(ax, crs=fluggs_sites.crs.to_string())
# optional parameters:
# source=ctx.providers.OpenStreetMap.Mapnik
# zoom=12
ax

In [ ]:
# Interactive map using folium

lngs = list(fluggs_sites['geometry'].apply(lambda coord: coord.x))
lats = list(fluggs_sites['geometry'].apply(lambda coord: coord.y))
avg_lat = sum(lats) / len(lats)
avg_lngs = sum(lngs) / len(lngs)

# folium uses (lat, long) or (y, x), respectively
locationlist = [[site.y, site.x] for site in fluggs_sites['geometry']]

m = folium.Map(location=[avg_lat, avg_lngs], zoom_start=10)
for point in range(0, len(locationlist)):
    popup = folium.Popup(folium.IFrame(html=fluggs_sites['site_name'][point], width=200, height=80))
    folium.Marker(locationlist[point], popup=popup).add_to(m)
m

# 3) Sensor data

### 3a) Get sensor data

In [ ]:
# Optional arguments for get_data(): procedures, phenomena, sites, begin, end
begin = '2019-01-01T00:00:00Z' 
end = '2019-01-31T23:59:59Z'
procedures = ['2m_Tiefe']
phenomena = ['Wassertemperatur']
sites = ['Opladen']

fluggs_obs = fluggs_sos.get_data(sites=sites, phenomena=phenomena, begin=begin, end=end)

fluggs_obs.head()

In [ ]:
# Check unique values in column
fluggs_obs['procedure'].unique()

In [ ]:
# Subset of data frame
#fluggs_obs[(fluggs_obs['site']=='Opladen') & (fluggs_obs['procedure']=='Einzelwerte')]
#fluggs_obs[(fluggs_obs['value'] > 5)] 

### 3b) Plot sensor data

#### Plot a single time series

In [ ]:
foi = sites[0]
x = fluggs_obs[fluggs_obs['site']==foi]['time_stamp'].to_numpy()
y = fluggs_obs[fluggs_obs['site']==foi]['value'].to_numpy()

plt.figure(figsize=(12,5))
plt.plot(x,y)
plt.xlabel('Time')
plt.ylabel(fluggs_obs['phenomenon'][0] + " " + fluggs_obs['procedure'][0] + " in " + fluggs_obs['unit'][0])
plt.title(fluggs_obs['site'][0])
plt.show()

#### Plot two time series of two different sites

In [ ]:
# Check data availability
fluggs_sos.get_data_availability(observedProperties=['Wassertemperatur'],procedures=['Einzelwerte'])

In [ ]:
# New request with two sites
begin = '2019-01-01T00:00:00Z' 
end = '2019-01-31T23:59:59Z'
procedures = ['Einzelwerte']
phenomena = ['Wassertemperatur']
sites = ['Opladen','Laaken']

fluggs_obs = fluggs_sos.get_data(sites=sites, phenomena=phenomena, begin=begin, end=end)
fluggs_obs.head()

In [ ]:
x1 = fluggs_obs[(fluggs_obs['site']==sites[0])]['time_stamp'].to_numpy()
y1 = fluggs_obs[(fluggs_obs['site']==sites[0])]['value'].to_numpy()

x2 = fluggs_obs[(fluggs_obs['site']==sites[1])]['time_stamp'].to_numpy()
y2 = fluggs_obs[(fluggs_obs['site']==sites[1])]['value'].to_numpy()

plt.figure(figsize=(12,5))
plt.plot(x1,y1,label=sites[0])
plt.plot(x2,y2,label=sites[1])
plt.xlabel('Time')
plt.ylabel(fluggs_obs['phenomenon'][0] + " " + fluggs_obs['procedure'][0] + " in " + fluggs_obs['unit'][0])
plt.legend()
plt.show()

#### Plot two time series of two different phenomena

In [ ]:
fluggs_sos.get_data_availability(featureOfInterest=['Bever-Talsperre'],observedProperties=['Luftfeuchte','Lufttemperatur'])

In [ ]:
# New request with two phenomena for one site
begin = '2011-01-01T00:00:00Z' 
end = '2011-01-31T23:59:59Z'
phenomena = ['Luftfeuchte','Lufttemperatur']
sites = ['Bever-Talsperre']

fluggs_obs = fluggs_sos.get_data(sites=sites, phenomena=phenomena, begin=begin, end=end)
fluggs_obs.head()

In [ ]:
x3 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[0])]['time_stamp'].to_numpy()
y3 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[0])]['value'].to_numpy()

x4 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[1])]['time_stamp'].to_numpy()
y4 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[1])]['value'].to_numpy()

fig, ax1 = plt.subplots(figsize=(12,5))

color = 'red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Luftfeuchte', color=color)
ax1.plot(x3, y3, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'blue'
ax2.set_ylabel('Lufttemperatur', color=color)  
ax2.plot(x4, y4, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.show()

#### Data sub-sampling

In [ ]:
interval = '4h'

# Phenomenon 1
df1 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[0])]
df1 = df1[['time_stamp','value']]

# Remove duplicate time stamps and sub-sample data
grouped = df1.groupby('time_stamp')['value']
grouped = grouped.agg('mean')
df1 = pd.DataFrame(grouped)
df1_sub = df1.resample(interval).nearest()
df1_sub = df1_sub.reset_index()


# Phenomenon 2
df2 = fluggs_obs[(fluggs_obs['phenomenon']==phenomena[1])]
df2 = df2[['time_stamp','value']]

# Remove duplicate time stamps and sub-sample data
grouped = df2.groupby('time_stamp')['value']
grouped = grouped.agg('mean')
df2 = pd.DataFrame(grouped)
df2_sub = df2.resample(interval).nearest()
df2_sub = df2_sub.reset_index()

In [ ]:
x3 = df1_sub['time_stamp'].to_numpy()
y3 = df1_sub['value'].to_numpy()

x4 = df2_sub['time_stamp'].to_numpy()
y4 = df2_sub['value'].to_numpy()

fig, ax1 = plt.subplots(figsize=(12,5))

color = 'red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Luftfeuchte', color=color)
ax1.plot(x3, y3, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'blue'
ax2.set_ylabel('Lufttemperatur', color=color)  
ax2.plot(x4, y4, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.show()

#### Histogram and Kernel density estimation (KDE)

In [ ]:
sns.distplot(y1, bins=10);
sns.distplot(y2, bins=10);

#### 2D KDE

In [ ]:
sns.kdeplot(y1,y2);

#### Joint plots/Correlation

In [ ]:
sns.jointplot(y1, y2, kind='reg', joint_kws={'line_kws':{'color':'red'}}).annotate(stats.pearsonr);

In [ ]:
sns.jointplot(y1, y2, kind='hex');

In [ ]:
sns.jointplot(y1, y2, kind='kde');